## KÜTÜPHANELERİMİZİ YÜKLÜYORUZ

In [2]:
import tensorflow as tf
import cv2
import numpy as np

## GPU KULLANIYORSAK, GPU'YU ÇAĞIRIYORUZ

In [3]:
#gpu kullanıyorsanız isteğe bağlı BEN KULLANMIYORUM
gpus=tf.config.experimental.list_logical_devices('GPU') #fiziksel cihazlardan CPU yu getirtiyoruz.
for gpu in gpus:  #Döngü ile gpu hafızası oldukça gpu yu kullandırtıyoruz.  
    tf.config.experimental.set_memory_growth(gpu,True)


## MODELİ TANITIYORUZ

In [4]:
from keras.models import load_model #model yükleme kütüphanemizi çağırıyoruz.
model= tf.saved_model.load('model') #Modelimizin yolunu veriyoruz
movenet=model.signatures['serving_default'] #varsayılan modelin imzasını belirterek, girdi ve çıktıları belirliyoruz.


## KENARLARI ÇİZİYORUZ

In [5]:
#0 Burun
#1 Sol göz
#2 Sağ göz 
#3 sol kulak
#4 sağ kulak
#5 sol omuz
#6 sağ omuz
#7 sol dirsek
#8 sağ dirsek
#9 sol bilek
#10 sağ bilek
#11 sol kalça
#12 sağ kalça
#13 sol diz
#14 sağ diz
#15 sol ayak bileği
#16 sağ ayak bileği

EDGES = {
    (0, 1): 'm', 
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

## BAĞLANTILARI ÇİZİYORUZ

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

## ANAHTAR NOKTALARI ÇİZİYORUZ

In [7]:
def draw_keypoints(frame, keypoints, confidence_threshold): #Anahtar nokta çizme fonksiyonumuza frame, anahtar nokta, güven değeri(Nokta değerleri) bilgilerini veriyoruz.
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [8]:
#işlev, algılanan ve işlenen her kişi arasında döngü oluşturur
def loop_through_people(frame,keypoints_with_scores,edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame,person,edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

## VİDEO ÖLÇÜ AYARI

Yükseklik/genişlik her ikisi de 32'nin katıdır.

Yükseklik-genişlik oranı, orijinal görüntünün en-boy oranını kapatmak için yakın (ve yeterlidir).

Büyük kenarı 256 yapın (hız/doğruluk gereksinimlerine göre bunu ayarlamanız gerekir). Örneğin, 720p bir görüntü (örn. 720x1280 (YxG)) yeniden boyutlandırılmalı ve 160x256 görüntüye doldurulmalıdır.

## KENDİ KAMERAMIZDA DENİYORUZ

In [17]:
cap= cv2.VideoCapture(0) #Kendi kameramızı tanıtıyoruz.
while cap.isOpened(): #While ile döngü içine alıp sürekli çalışmasını sağlıyoruz. Kendi kameramız sürekli açık iken aşağıdaki işlemleri yaptırıyoruz.
    ret, frame=cap.read() #kameradan okunan veriyi, dönen veri ve frame olarak belirliyoruz.
    #resize image 
    img= frame.copy() #Frame'imizin kopyasını alıyoruz ve tespit yapıpı ölçülendirmek için kullanıyoruz.
    img=tf.image.resize_with_pad(tf.expand_dims(img,axis=0),192,256) #model 32 bit olduğu için 32 nin katlarını ölçü olarak kullanıyor.
                                                                     #Model ölçüsünü ne kadar büyütürsek, hafızayı o kadar çok kullanır.
    input_img = tf.cast(img, dtype=tf.int32) #32 bite dönüştürüyoruz.

    #Detection section
    results=movenet(input_img) #32 bitlik verimizi movenet() fonksiyonumuzun içine alıp , bunu sonuç olarak tanımlıyoruz.
    keypoints_with_scores=results['output_0'].numpy()[:,:,:51].reshape((6,17,3)) #Anahtar noktaları nupmy ile diziye alıp, results daki 51 verinin hepsini alıyoruz.
                                  #Reshape ile 6 insan alacağını 17 anahtar nokta alacağını, her anahtar için 3 değer alacağını belirliyoruz.
    
    #rendeer keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.2)
    
    cv2.imshow('Movenet Multipose Detection', frame) #Çerçeve başlığımızı belirliyoruz.

    if cv2.waitKey(10) & 0xFF==ord('q'): #Q tuşuna 10ms basılınca döngünün bitmesini ayarlıyoruz.
        break #Döngüyü kırma
cap.release() #Video ile işimizin bittiğini, videonun serbest bırakılmasını belirtiyoruz.
cv2.destroyAllWindows() #Bütün ekranda Dosyamızın kapatılmasını gösteriyoruz.

## VİDEO ÜZERİNDEN DENİYORUZ

In [25]:
import cv2
import tensorflow as tf

# Video yakalama
cap = cv2.VideoCapture("hiker.mov")

# Yeni boyut değerleri
new_width = 1920  # Yeni genişlik değeri
new_height = 1080  # Yeni yükseklik değeri

while cap.isOpened():
    ret, frame = cap.read()

    # Görüntüyü yeniden boyutlandırma
    resized_frame = cv2.resize(frame, (new_width, new_height))

    img = resized_frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352, 640)
    input_img = tf.cast(img, dtype=tf.int32)

    # Detection bölümü (bu kısmı değiştirmemeniz gerekebilir)
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:, :, :51].reshape((6, 17, 3))

    # Anahtar nokta ayarları (bu kısmı değiştirmemeniz gerekebilir)
    loop_through_people(resized_frame, keypoints_with_scores, EDGES, 0.2)

    cv2.imshow('Movenet Multipose Detection', resized_frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
